# Combining Pandas Objects

- Appending new rows to DataFrames
- Concatenating multiple DataFrames together
- Comparing President Trump's and Obama's approval ratings
- Understanding the differences between concat, join, and merge

A wide variety of options are available to combine two or more DataFrames or Series
together. The **append** method is the least flexible and only allows for new rows to be
appended to a DataFrame. The **concat** method is very versatile and can combine any
number of DataFrames or Series on either axis. The **join** method provides fast lookups by
aligning a column of one DataFrame to the index of others. The **merge** method provides
SQL-like capabilities to join two DataFrames together. 

In [93]:
# import packages we need for exploratory data analysis (EDA)
# to store tabular data
import pandas as pd
# to do some math
import numpy as np
# a popular data visualization tool
import matplotlib.pyplot as plt
# another popular data visualization tool
import seaborn as sns
# allows the notebook to render graphics
%matplotlib inline
# a popular data visualization theme
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [94]:
path = 'D:\\ml_code\\data_science\\pandas\\Pandas_Code\\data\\'
def get_df(csv_name):
    df = pd.read_csv(path+csv_name)
    return df

## Appending new rows to DataFrames

In [95]:
names = get_df('names.csv')
names

,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2


In [96]:
#create a list that contains some new data and use the .loc indexer to set a single row label equal to this new data:
new_data_list = ['Aria',2]
names.loc[4] = new_data_list
names

,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2
4,Aria,2


In [97]:
#The .loc indexer uses labels to refer to the rows. In this case, the row labels
#exactly match the integer location. It is possible to append more rows with noninteger labels:
names.loc['five'] = ['Zach',3]
names

,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2
4,Aria,2
five,Zach,3


The .loc indexing operator is used to select and assign data based on the row and column
labels. The first value passed to it represents the row label. 

This operation modifies the names DataFrame itself. If there
was a previously existing row with a label equal to the integer 4, this command would have
written over it. This modification in-place makes this indexing operator riskier to use than
the append method, which never modifies the original calling DataFrame.

## Concatenating multiple DataFrames together

The concat function enables concatenating two or more DataFrames (or Series)
together, both vertically and horizontally. As per usual, when dealing with multiple pandas
objects simultaneously, concatenation doesn't happen haphazardly but aligns each object by
their index.


In [98]:
#Combine DataFrames both horizontally and vertically with the concat function and then change the
#parameter values to yield different results.

#Read in the 2016 and 2017 stock datasets, and make their ticker symbol the index:
stocks_2016 = pd.read_csv(path+'stocks_2016.csv',index_col='Symbol')
stocks_2017 = pd.read_csv(path+'stocks_2017.csv',index_col='Symbol')

In [99]:
stocks_2016

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70


In [100]:
stocks_2017

,Shares,Low,High
Symbol,,,
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85
TXN,500,15,23
TSLA,100,100,300


In [101]:
s_list = [stocks_2016,stocks_2017]
pd.concat(s_list)

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85
TXN,500,15,23
TSLA,100,100,300


By default, the concat function concatenates DataFrames vertically, one on top
of the other. One issue with the preceding DataFrame is that there is no way to
identify the year of each row. The concat function allows each piece of the
resulting DataFrame to be labeled with the keys parameter. This label will
appear in the outermost index level of the concatenated frame and force the
creation of a MultiIndex. Also, the names parameter has the ability to rename
each index level for clarity

In [102]:
pd.concat(s_list,keys=['2016','2017'],names=['Year','Symbol'])

Shares  Low  High
Year Symbol                   
2016 AAPL        80   95   110
     TSLA        50   80   130
     WMT         40   55    70
2017 AAPL        50  120   140
     GE         100   30    40
     IBM         87   75    95
     SLB         20   55    85
     TXN        500   15    23
     TSLA       100  100   300

In [103]:
#concatenate horizontally by changing the axis parameter to columns or 1:
pd.concat(s_list, keys=['2016', '2017'],axis='columns', names=['Year', None])

Year   2016                2017              
     Shares   Low   High Shares    Low   High
AAPL   80.0  95.0  110.0   50.0  120.0  140.0
GE      NaN   NaN    NaN  100.0   30.0   40.0
IBM     NaN   NaN    NaN   87.0   75.0   95.0
SLB     NaN   NaN    NaN   20.0   55.0   85.0
TSLA   50.0  80.0  130.0  100.0  100.0  300.0
TXN     NaN   NaN    NaN  500.0   15.0   23.0
WMT    40.0  55.0   70.0    NaN    NaN    NaN

Missing values appear whenever a stock symbol is present in one year
but not the other. The concat function, by default, uses an outer join, keeping all
rows from each DataFrame in the list. However, it gives us options to only keep
rows that have the same index values in both DataFrames. This is referred to as
an inner join. We set the join parameter to inner to change the behavior

In [104]:
pd.concat(s_list, join='inner', keys=['2016', '2017'],axis='columns', names=['Year', None])


Year     2016            2017          
       Shares Low High Shares  Low High
Symbol                                 
AAPL       80  95  110     50  120  140
TSLA       50  80  130    100  100  300

## Understanding the differences between concat, join, and merge

### concat:
- Pandas function
- Combines two or more pandas objects vertically or horizontally
- Aligns only on the index
- Errors whenever a duplicate appears in the index
- Defaults to outer join with option for inner

### join:
- DataFrame method
- Combines two or more pandas objects horizontally
- Aligns the calling DataFrame's column(s) or index with the other objects' index (and not the columns)
- Handles duplicate values on the joining columns/index by performing a cartesian product
- Defaults to left join with options for inner, outer, and right

### merge:
- DataFrame method
- Combines exactly two DataFrames horizontally
- Aligns the calling DataFrame's column(s)/index with the other
- DataFrame's column(s)/index
- Handles duplicate values on the joining columns/index by performing a cartesian product
- Defaults to inner join with options for left, outer, and right

Read in stock data for 2016, 2017, and 2018 into a list of DataFrames using a
loop instead of three different calls to the read_csv function. Jupyter notebooks
currently only allow a single DataFrame to be displayed on one line. However,
there is a way to customize the HTML output with help from the IPython
library. The user-defined display_frames function accepts a list of DataFrames
and outputs them all in a single row

In [105]:
from IPython.display import display_html

In [106]:
years = 2016,2017,2018
stock_tables = [pd.read_csv(path+'stocks_{}.csv'.format(year),index_col='Symbol')for year in years]

In [107]:
def display_frames(frames, num_spaces=0):
    t_style = '<table style="display: inline;"'
    tables_html = [df.to_html().replace('<table', t_style)for df in frames]
    space = '&nbsp;' * num_spaces
    display_html(space.join(tables_html), raw=True)

In [108]:
display_frames(stock_tables,30)

,Shares,Low,High
Symbol,,,
AAPL,80,95,110
TSLA,50,80,130
WMT,40,55,70
,Shares,Low,High
Symbol,,,
AAPL,50,120,140
GE,100,30,40
IBM,87,75,95
SLB,20,55,85


In [109]:
stocks_2016, stocks_2017, stocks_2018 = stock_tables

The concat function is the only one able to combine DataFrames vertically. Let's
do this by passing it the list stock_tables:

In [110]:
pd.concat(stock_tables,keys=[2016,2017,2018])

Shares  Low  High
     Symbol                   
2016 AAPL        80   95   110
     TSLA        50   80   130
     WMT         40   55    70
2017 AAPL        50  120   140
     GE         100   30    40
     IBM         87   75    95
     SLB         20   55    85
     TXN        500   15    23
     TSLA       100  100   300
2018 AAPL        40  135   170
     AMZN         8  900  1125
     TSLA        50  220   400

In [111]:
#It can also combine DataFrames horizontally by changing the axis parameter to columns:

pd.concat(stock_tables,keys=[2016,2017,2018],axis='columns')

2016                2017                 2018               
     Shares   Low   High Shares    Low   High Shares    Low    High
AAPL   80.0  95.0  110.0   50.0  120.0  140.0   40.0  135.0   170.0
AMZN    NaN   NaN    NaN    NaN    NaN    NaN    8.0  900.0  1125.0
GE      NaN   NaN    NaN  100.0   30.0   40.0    NaN    NaN     NaN
IBM     NaN   NaN    NaN   87.0   75.0   95.0    NaN    NaN     NaN
SLB     NaN   NaN    NaN   20.0   55.0   85.0    NaN    NaN     NaN
TSLA   50.0  80.0  130.0  100.0  100.0  300.0   50.0  220.0   400.0
TXN     NaN   NaN    NaN  500.0   15.0   23.0    NaN    NaN     NaN
WMT    40.0  55.0   70.0    NaN    NaN    NaN    NaN    NaN     NaN

Now that we have started combining DataFrames horizontally, we can use the
**join** and **merge** methods to replicate this functionality of **concat**. Here, we use
the **join** method to combine the stock_2016 and stock_2017 DataFrames. By
default, the DataFrames align on their index. If any of the columns have the same
names, then you must supply a value to the lsuffix or rsuffix parameters to
distinguish them in the result

In [112]:
stocks_2016.join(stocks_2017,lsuffix='_2016',rsuffix='_2017',how='outer')

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017
Symbol,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0
GE,NaN,NaN,NaN,100.0,30.0,40.0
IBM,NaN,NaN,NaN,87.0,75.0,95.0
SLB,NaN,NaN,NaN,20.0,55.0,85.0
TSLA,50.0,80.0,130.0,100.0,100.0,300.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0
WMT,40.0,55.0,70.0,NaN,NaN,NaN


Lets turn to **merge** that, unlike **concat** and **join**, can combine exactly two
DataFrames together. By default, **merge** attempts to align the values in the
columns that have the same name for each of the DataFrames. However, you can
choose to have it align on the index by setting the boolean parameters
left_index and right_index to True. Let's merge the 2016 and 2017 stock
data together

In [113]:
stocks_2016.merge(stocks_2017,left_index=True,right_index=True)

,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
Symbol,,,,,,
AAPL,80,95,110,50,120,140
TSLA,50,80,130,100,100,300


In [114]:
#By default, merge uses an inner join and automatically supplies suffixes for
#identically named columns. Let's change to an outer join and then perform
#another outer join of the 2018 data to exactly replicate concat

step1 = stocks_2016.merge(stocks_2017, left_index=True,right_index=True, how='outer',suffixes=('_2016', '_2017'))
stock_merge = step1.merge(stocks_2018.add_suffix('_2018'),left_index=True, right_index=True,how='outer')

In [115]:
stock_merge

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
Symbol,,,,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0
GE,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TSLA,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
WMT,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN


Turn our comparison to datasets where we are interested in aligning
together the values of columns and not the index or column labels themselves.
The merge method is built exactly for this situation. Let's take a look at two new
small datasets, food_prices and food_transactions:

In [116]:
names = ['prices', 'transactions']
food_tables = [pd.read_csv(path+'food_{}.csv'.format(name))for name in names]
food_prices, food_transactions = food_tables
display_frames(food_tables, 30)

,item,store,price,Date
0,pear,A,0.99,2017
1,pear,B,1.99,2017
2,peach,A,2.99,2017
3,peach,B,3.49,2017
4,banana,A,0.39,2017
5,banana,B,0.49,2017
6,steak,A,5.99,2017
7,steak,B,6.99,2017
8,steak,B,4.99,2015
,custid,item,store,quantity


In [120]:
#. If we wanted to find the total amount of each transaction, we would need to join
#these tables on the item and store columns:

food_transactions = food_transactions.merge(food_prices, on=['item', 'store'])
food_transactions

,custid,item,store,quantity,price,Date
0,1,pear,A,5,0.99,2017
1,1,banana,A,10,0.39,2017
2,2,steak,B,3,6.99,2017
3,2,steak,B,3,4.99,2015
4,2,steak,B,1,6.99,2017
5,2,steak,B,1,4.99,2015
6,2,pear,B,1,1.99,2017
7,2,peach,B,2,3.49,2017
